Project: /_project.yaml
Book: /_book.yaml
Keywords: examples,gemini,beginner,googleai,quickstart,python,text,chat,vision,embed
Image_path: /static/site-assets/images/docs/logo-python.svg

<link rel="stylesheet" href="/site-assets/css/style.css">

# Gemini API: Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/text_quickstart"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Setup



### Install the Python SDK

The Python SDK for the Gamini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:


In [ ]:
!pip install -U google-generativeai

### Import packages

Import the necessary packages.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in MakerSuite.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`



In [3]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY')

genai.configure(api_key=API_KEY)

## Function calls

The `google.ai.generativelanguage` client library providses access to the low level types required for function calling.

In [4]:
import google.ai.generativelanguage as glm

A `glm.Tool` contains a list of `glm.FunctionDeclarations`. These just describe the function, they don't implement it.

In [5]:
datetime = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='datetime',
        description="Returns the current UTC date and time."
      )
    ]
)

Pass a list of tools to the `genai.GenerativeModel` constructor to give the model access:

In [6]:
model = genai.GenerativeModel(
    'gemini-pro',
    tools=[datetime])

For this basic tools support use chat-mode since tools require multiple rounds of back and forth.

In [7]:
chat = model.start_chat()

response = chat.send_message(
    'How many days until Christmas',
)

When the model needs to call a tool to answer a question it returns a `glm.Part` containing a `function_call` instead of a `text` attribute:

In [8]:
response.candidates

[content {
  parts {
    function_call {
      name: "datetime"
      args {
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
]

Reply with a `glm.Part` containing a `glm.FunctionResponse` to allow the model to funish the answer:

In [ ]:
response = chat.send_message(
  glm.Content(
    parts=[glm.Part(
        function_response = glm.FunctionResponse(
          name='datetime',
          response={'datetime': 'Sun Dec 5 03:33:56 PM UTC 2023'}
        )
    )]
  )
)

The model may respond with either a `text` response or another `glm.FunctionCall`:

In [ ]:
response.text

' Christmas is on Sunday Dec 25, 2023. So, there are 20 days until Christmas.'

That `datetime` tool only contained a single function, which takes no arguments. Next try something more complex.

LLM's are bad at arithmetic:


In [22]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat()

a = 2312371
b = 234234

response = chat.send_message(
    f"What's {a} X {b} ?",

)
print(response.text)

5447805529729


In [23]:
a*b

541635908814

Sometimes it's a fraction of 1% off, sometimes it's off by 10X.

In [25]:
error_percent = (a*b - int(response.text.replace(',', '')))/(a*b) * 100

print(f"Error: {error_percent:.2f}%")

Error: -905.81%


So, describe a calculator as a `glm.Tool`:

In [31]:
calculator = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='add',
        description="Returns sum of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a': glm.Schema(type=glm.Type.NUMBER),
                'b': glm.Schema(type=glm.Type.NUMBER)
            }
        )
      ),
      glm.FunctionDeclaration(
        name='multiply',
        description="Returns product of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a':glm.Schema(type=glm.Type.NUMBER),
                'b':glm.Schema(type=glm.Type.NUMBER)
            }

        )
      )
    ])

Give the model the calculator and ask again:

In [32]:
model = genai.GenerativeModel('gemini-pro', tools=[calculator])
chat = model.start_chat()

response = chat.send_message(
    "What's 2312371 X 234234 ?",
)

In [33]:
response.candidates

[content {
  parts {
    function_call {
      name: "multiply"
      args {
        fields {
          key: "a"
          value {
            number_value: 2312371.0
          }
        }
        fields {
          key: "b"
          value {
            number_value: 234234.0
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
]